# Distribution computation for multiple manipluated dices

Computing the distribution allows to modell **game design' specific payouts**.
A fixed number of dices, which represent the independent events, are considered. Additionally, each dice has a fixed number of outcomes, which can be weighted. In this sense manipulated dices means:
 - dice can have more than six outcomes
 - the outcomes can be weighted
 
 
 - Part 3: A general formula for a fixed number of equally-distributed outcomes for each dice. Creating a game' payout based on the distribution
 - Part 4: Cycle evaluation for a fixed number of events with weighted-outcomes in C# accessed via REST-API



# Imports

In [14]:
import sys
sys.path.insert(0, 'lib\\')
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import scipy.special

# A general formula for a fixed number of events with equally-distributed outcomes
Part 3: Deriving a formula for computing the hits and the probabilities for a fixed number of dices, having a fixed number of equally-distributed outcomes.

Let numOutcomes be the number of outcomes for each event and let numEvent be the number of events resp. dices. 

 - The total cycle represents all combinations and can be evaluated by:

$$ totalCycle = {numOutcomes}^{numEvents}$$

For example, having 9 dices and each dice has 10 different outcomes leads to a total cycle of:

In [45]:
numEvents = 9
numOutcomes = 10
totalCycle = numOutcomes**numEvents
print(f"{totalCycle:,}")

1,000,000,000


The number of hits, denoted by k, can be evaluated by:
 
$$hits(k) = \binom{numEvents}{k} * {(numOutcomes-1)}^{(numEvents - k)}$$

--- 
This fact derives out of the following:
 1. Firstly, the number of hits resp. correct scores out of a given number of events is determined by the binomial coefficient. For example, having 2 hits on 9 dices with 10 outcomes leads to 36 different hit-combinations:

$$\binom{9}{2} = 36$$


 2. Secondly, for each hit-combination, the other dices need to cycle through all no-hit-combinations i.e., which do not lead to another hit. The number of no-hits on a dice is simply (numOutcomes - 1) and the number of dices, which show no-hit is (numEvent - k). Continuing with the example determines 7 no-hits dices and, on each 9 outcomes with no-hit:

$${(10-1)}^{(9 - 2)} = {(9)}^{(7)} = 4,782,969 $$


 3. Thirdly, the product determines the total success-hits within the cycle. For 2 hits with 9 dices having 10 outcomes:
   
$$\binom{9}{2} * {(10-1)}^{(9 - 2)} = 36 * 4,782,969 = 172,186,884 $$

---

Summing up all hits, for k = 0,...,numEvents; leads to

$$ totalCycle = {numOutcomes}^{numEvents} = \sum_{k=0}^{numEvents} \binom{numEvents}{k} * {(numOutcomes-1)}^{(numEvents - k)}$$ 


The probability of having k hits is then simply the ratio:

$$P(X=k) = \frac{hits(k)}{totalCycle} = \frac{\binom{numEvents}{k} * {(numOutcomes-1)}^{(numEvents - k)}}{{numOutcomes}^{numEvents}}$$
  

Continuing with the example, the probability of having 2 hits wiht 9 dices with 10 outcomes:
  
$$P(X=2) = \frac{\binom{9}{2} * {(10-1)}^{(9 - 2)}}{{10}^{9}} = 0.172186884 $$

In [95]:
temp = [[0 for x in range(numEvents+1)] for y in range(5)]
acc_hits = 0

for i in range(numEvents+1):
    temp[0][i] = scipy.special.binom(numEvents, i)
    temp[1][i] = (numOutcomes-1)**(numEvents - i)
    temp[2][i] = temp[0][i] * temp[1][i]
    acc_hits = acc_hits + temp[2][i] 
    temp[3][i] = acc_hits
    temp[4][i] = temp[2][i] *1.0 / totalCycle * 1.0

pd.options.display.float_format = '{:,.0f}'.format  
df = pd.DataFrame(temp).transpose().rename(columns={0: "Number of different combinations of hits, which lead to k hits", 
                                               1: "Number of different combinations of no-hits",
                                               2: "Total hits for k",
                                               3: "Accumulated hits", 
                                               4: "P(X)"})
df['P(X)'] = df['P(X)'].map('{:,.9f}'.format)
df

,"Number of different combinations of hits, which lead to k hits",Number of different combinations of no-hits,Total hits for k,Accumulated hits,P(X)
0,1,"387,420,489","387,420,489","387,420,489",0.387420489
1,9,"43,046,721","387,420,489","774,840,978",0.387420489
2,36,"4,782,969","172,186,884","947,027,862",0.172186884
3,84,"531,441","44,641,044","991,668,906",0.044641044
4,126,"59,049","7,440,174","999,109,080",0.007440174
5,126,"6,561","826,686","999,935,766",0.000826686
6,84,729,"61,236","999,997,002",0.000061236
7,36,81,"2,916","999,999,918",0.000002916
8,9,9,81,"999,999,999",0.000000081
9,1,1,1,"1,000,000,000",0.000000001


## Creating a game specific payout based on the distribution

The goal of the set-up of game is it to award a specific win. The game's stake is 1 and **return-to-player** should be <100%.

In [96]:
payouts = [0.0,0.0,1.0,5.0,50.0,100.0,1000.0,10000.0,100000.0,1000000.0]
df['Payouts'] = payouts
df

,"Number of different combinations of hits, which lead to k hits",Number of different combinations of no-hits,Total hits for k,Accumulated hits,P(X),Payouts
0,1,"387,420,489","387,420,489","387,420,489",0.387420489,0
1,9,"43,046,721","387,420,489","774,840,978",0.387420489,0
2,36,"4,782,969","172,186,884","947,027,862",0.172186884,1
3,84,"531,441","44,641,044","991,668,906",0.044641044,5
4,126,"59,049","7,440,174","999,109,080",0.007440174,50
5,126,"6,561","826,686","999,935,766",0.000826686,100
6,84,729,"61,236","999,997,002",0.000061236,"1,000"
7,36,81,"2,916","999,999,918",0.000002916,"10,000"
8,9,9,81,"999,999,999",0.000000081,"100,000"
9,1,1,1,"1,000,000,000",0.000000001,"1,000,000"


For each hit-category a fictious payout based on the game's distribution is defined. The **hold** resp. margin, which is defined by 1-RTP is achieved by downgrading each hit-category further. 

The RTP is then simply defined by:


$$ RTP = \sum_{k=0}^{numEvents} Payouts(k) * P(X=k)$$   


In [97]:
df['RTP of k hits'] = df['Payouts'].astype(float) * df['P(X)'].astype(float)
df['RTP of k hits'] = df['RTP of k hits'].map('{:,.9f}'.format)
df
print('RTP =', df['RTP of k hits'].astype(float).sum())

RTP = 0.949565404


# Cycle evaluation for a fixed number of events with weighted-outcomes in C# accessed via REST-API
Part 4: .. .

...

...
..